In [3]:
import pandas as pd
import duckdb

In [4]:
capteurs = pd.read_csv("data/capteur_a_retrail.csv")
shuffled = capteurs.sample(len(capteurs))

## LAG: la syntaxe

In [7]:
query = """
SELECT *,
LAG(visiteurs_count) OVER() AS ligne_precedente
FROM shuffled
LIMIT 8
"""
df = duckdb.sql(query).df()
df

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,ligne_precedente
0,2023-08-03,porte_a,4400.0,5,4920.0,3936.0,NaN
1,2023-08-04,porte_a,5500.0,6,4920.0,3936.0,4400.0
2,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5500.0
3,2023-08-22,porte_a,4900.0,3,4920.0,3936.0,6000.0
4,2023-09-04,porte_a,4500.0,2,4700.0,3760.0,4900.0
5,2023-08-15,porte_a,4200.0,3,4920.0,3936.0,4500.0
6,2023-08-28,porte_a,4300.0,2,4920.0,3936.0,4200.0
7,2023-08-29,porte_a,4750.0,3,4920.0,3936.0,4300.0


## LAG after WHERE

On pourrait s'attendre à ce que le LAG soit opéré sur l'ensemble de la table, <br />
et que le WHERE vienne ensuite filtrer et rendre l'interprétation difficile.

Mais ce n'est pas ce qui se passe :

In [8]:
query = """
SELECT *,
LAG(visiteurs_count) OVER() AS ligne_précédente
FROM shuffled
WHERE weekday = 7
"""
df = duckdb.sql(query).df()
df

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,ligne_précédente
0,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,NaN
1,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.0
2,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,6000.0
3,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,4000.0
4,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,5500.0


On peut constater que le LAG est opéré uniquement sur les valeurs de notre subset !

Ordre des opérations présenté par SQL Server: <br />
https://learn.microsoft.com/en-us/sql/t-sql/queries/select-transact-sql?view=sql-server-2017#logical-processing-order-of-the-select-statement

En théorie, les window functions sont dans le SELECT, donc après le WHERE ;)

## LAG sur toute la dataframe

Exercice: mettez, en face de chaque "visiteurs_count", <br />
le visiteurs_count du même jour de la semaine précédente

In [10]:
%load solutions/13a_lag.py

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,lag_visiteurs_count
0,2023-08-03,porte_a,4400.0,5,4920.0,3936.0,NaN
1,2023-08-10,porte_a,4400.0,5,4920.0,3936.0,4400.0
2,2023-08-17,porte_a,3600.0,5,4920.0,3936.0,4400.0
3,2023-08-24,porte_a,4700.0,5,4920.0,3936.0,3600.0
4,2023-08-31,porte_a,4700.0,5,4920.0,3936.0,4700.0
5,2023-08-07,porte_a,4200.0,2,4920.0,3936.0,NaN
6,2023-08-14,porte_a,3900.0,2,4920.0,3936.0,4200.0
7,2023-08-21,porte_a,4300.0,2,4920.0,3936.0,3900.0
8,2023-08-28,porte_a,4300.0,2,4920.0,3936.0,4300.0
9,2023-09-04,porte_a,4500.0,2,4700.0,3760.0,4300.0


Exercice: utilisez la même requête pour calculer la différence entre ces deux colonnes, <br />
Voire même, si vous êtes motivés, le pourcentage de changement d'une semaine sur l'autre <br /> 
(toujours pour le même jour)

In [ ]:
%load solutions/13pct_of_evolution.py